# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [87]:
#Your code here
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [5]:
df.shape

(8188, 4)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


Hints:

* Start investigating the id column:
    * How many viewers also clicked?
    * Are there any anomalies with the data; did anyone click who didn't view?
    * Is there any overlap between the control and experiment groups? 
        * If so, how do you plan to account for this in your experimental design?

In [54]:
df.id.nunique()

6328

There are 6328 unique IDs in our dataset, meaning that there are repeated IDs across multiple rows. My assumption is that the repeated rows are for clicks from the same user id's that also, subsequently, are counted as having viewed the site. If there are no null values and the number of unique id's is the same as the number of views then this confirms this scenario.

In [55]:
df.isnull().sum()

timestamp    0
id           0
group        0
action       0
dtype: int64

In [24]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [33]:
cids = set(df[df['action'] == 'click']['id'].unique())
vids = set(df[df['action']=='view']['id'].unique())
print("Number of Viewers: {} \tNumber of Clickers: {}".format(len(vids), len(cids)))
print("Number of Viewers who did NOT click: {}".format(len(vids-cids)))
print("Number of Clickers who did NOT view: {}".format(len(cids-vids))) # this shouldn't be possible, but just confirm

Number of Viewers: 6328 	Number of Clickers: 1860
Number of Viewers who did NOT click: 4468
NUmber of Clickers who did NOT view: 0


As we suspected, there are no clickers who did not view. No anomalies here.

If view-to-click conversion is clicks over views, then...

In [56]:
print("Overall click conversion: {}".format(1860/6328))

Overall click conversion: 0.2939317319848293


In [34]:
# let's look at the control and experiment groups within clickers and viewers
exp_ids = set(df[df.group == 'experiment']['id'].unique())
ctrl_ids = set(df[df.group == 'control']['id'].unique())
print("Overlap of experiment and control groups (ids): {}".format(len(exp_ids&ctrl_ids)))

Overlap of experiment and control groups (ids): 0


Thankfully, there is not overlap between the experiment and control groups.

In [37]:
exp_df = df.loc[df.group=="experiment"]
exp_df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view


In [39]:
exp_df.shape

(3924, 4)

In [40]:
ctrl_df = df.loc[df.group=='control']
print(ctrl_df.shape)
ctrl_df.head()

(4264, 4)


,timestamp,id,group,action
3,2016-09-24 19:59:02.646620,671993,control,view
9,2016-09-24 21:00:12.278374,560027,control,view
23,2016-09-25 00:25:14.141290,281985,control,view
24,2016-09-25 01:14:48.603202,407864,control,view
25,2016-09-25 02:16:11.046654,342984,control,view


In [44]:
exp_df.action.value_counts()

view     2996
click     928
Name: action, dtype: int64

In [46]:
ctrl_df.action.value_counts()

view     3332
click     932
Name: action, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

**Initial Hypotheses**
- $H_0$: avg_clicks_expr =< avg_clicks_ctrl
- $H_1$: avg_clicks_expr > avg_clicks_ctrl

**Parameters and Metrics** 
- $\alpha = 0.05$
- run a Welch's t-test on mean clicks between experiment and control groups (should be one-sided)


In [59]:
#Your code here
ctrl_n = len(ctrl_df)
exp_n = len(exp_df)


In [73]:
ctrl_df['count'] = 1
exp_df['count'] = 1

/Users/serenaquiroga/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/serenaquiroga/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

ctrl_df['count'] = 1
exp_df['count'] = 1

In [79]:
control = ctrl_df.pivot(index='id', columns='action', values='count')
experiment = exp_df.pivot(index='id', columns='action', values='count')

#since we separated out the action items into separate columns, we'll fill the null values with 0
control.fillna(value=0, inplace=True)
experiment.fillna(value=0, inplace=True)

action
click    0
view     0
dtype: int64

In [83]:
#Let's extract the metrics we care about
print("Sample Sizes: \nControl: {} \tExperiment: {}".format(ctrl_n, exp_n))
print("Total Clicks: \nControl: {} \tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Avg Click Rate: \nControl: {} \tExperiment: {}".format(control.click.mean(), experiment.click.mean()))

Sample Sizes: 
Control: 4264 	Experiment: 3924
Total Clicks: 
Control: 932.0 	Experiment: 928.0
Avg Click Rate: 
Control: 0.2797118847539016 	Experiment: 0.3097463284379172


In [85]:
import flatiron_stats as fs

In [88]:
t, p = stats.ttest_ind(control.click, experiment.click, equal_var=False)
print(t)
print(p/2) # because this should be a one-sided p-value

-2.615440020788211
0.004466402814337101


In [90]:
#now to verify with our own function
fs.p_value_welch_ttest(control.click, experiment.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [91]:
#Your code here
control_rate = control.click.mean()
expected_expr_clicks_under_null = control_rate * len(experiment)
print(expected_expr_clicks_under_null)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [93]:
#Your code here
actual_expr_clicks = experiment.click.sum()

n = len(experiment)
p = control_rate #expected probability
var = n*p*(1-p)
std = np.sqrt(var)

z_score = (actual_expr_clicks - expected_expr_clicks_under_null) / std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [94]:
#Your code here
p_val = 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: This p-value is slightly lower than that calculated using the Welch's t-test, but both are low enough to safely reject the null hypothesis. The experimental group appears to have a higher click rate on average than the control group.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.